Enter parameters here

In [ ]:
import numpy as np
from scipy import integrate
import pandas as pd
from generateTestPositions import generateTestPositions

In [ ]:
# Testing with dummy data...only required when data is not available for code testing...generates projections based on magnification, rotation and known velocity profile
NumOfDataPoints = 1
clusterness = 0.1 # smaller number the more clustered
# Input conditions
initial_positions = []
v = []
# for i in range(1):
#     [x,y,z] = get_initial_position(i)
#     initial_positions.append([x, y, z])
#     vel = get_velocity_function(i)
#     v.append(vel)
initial_positions = np.zeros((NumOfDataPoints,3))
initial_positions[0] = [6,4,8]
v.append(lambda t: [3*np.sin(t), 2*np.cos(t), np.sin(t)])
# v.append(lambda t: [4,2,2])
noise = 1e-3
theta_degrees = 1.8
rev = 2  # revolutions of camera for the entire process
NOS = int(rev * 360 / theta_degrees)
NOS = 200
NOS_per_section = 159  # must be larger than 5 to satisfy equations
camera_speed = 0.5  # in Hz or revolution per second
SOD = 38  # mm, Source-Reference Distance
ODD = 462  # mm, Reference-Detector (screen) Distance
radius = 1

pixelResolution = 0.172  # every pixel is equal to mm
method = 'acceleration'
dataPiling = 'overlap'

# Auto-calculations of the rest of the parameters derived from the setting above
delta_T = camera_speed * theta_degrees / 360
shots_per_second = 1 / delta_T

# Define the velocity function
# v = lambda t: [0.9 * np.sin(t), 0.9 * np.cos(t), 1]

# Pack conditions into a list
conditions = [noise, delta_T, NOS, theta_degrees, NOS_per_section, SOD, ODD,method,dataPiling]
print(conditions)

xz_proj = np.zeros((NOS, NumOfDataPoints*2))
real_positions = np.zeros((NOS, NumOfDataPoints*3))
# Generate test positions
for i in range(NumOfDataPoints):
    vel = v[i]
    xz_proj[:,i*2:i*2+2], real_positions[:,i*3:i*3+3]= generateTestPositions(vel, initial_positions[i], conditions)